<a href="https://colab.research.google.com/github/dathong/ProtoryNet/blob/master/ProtoryNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
#import
!pip install https://github.com/scikit-learn-contrib/scikit-learn-extra/archive/master.zip
from sklearn_extra.cluster import KMedoids
import pandas as pd
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import nltk, re, time
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import namedtuple
import os
import pickle



  Using cached https://github.com/scikit-learn-contrib/scikit-learn-extra/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.1.0b2-cp36-cp36m-linux_x86_64.whl size=342229 sha256=6b74d77b49daff5b28eaf8b4f05bd44dd3d907a6511097738f9f02bcc3c2e4a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-w4462d7l/wheels/d3/a5/a8/411bc2d0939f2cc9d17f34f0d3457043c68e14a98b91fd8301
Successfully built scikit-learn-extra
1.15.2


In [165]:
print(tf.__version__)
from google.colab import drive 
drive.mount('/content/drive')

1.15.2
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
#load data
print('----preparing data------')

with open ("/content/drive/My Drive/deep_learning/dl_research/interpretable_RNN/amazon/train_vects", 'rb') as fp:
  train_vects = pickle.load(fp)

with open ("./drive/My Drive/deep_learning/dl_research/interpretable_RNN/amazon/y_train", 'rb') as fp:
  y_train = pickle.load(fp)

with open ('./drive/My Drive/deep_learning/dl_research/interpretable_RNN/amazon/train_clean', 'rb') as fp:
    train_clean = pickle.load(fp)

with open ('./drive/My Drive/deep_learning/dl_research/interpretable_RNN/amazon/train_not_clean', 'rb') as fp:
    train_not_clean = pickle.load(fp)

y_train_full = np.array(y_train)
y_train = np.array(y_train)
y_train = np.expand_dims(y_train, axis=1)


----preparing data------


In [0]:
# print(len(train_vects),len(train_vects[0]))
# print(len(y_train),y_train[:10])

In [167]:
data_size = 3000
# valid_size = 1000
def get_batches(x, y,batch_size):
    '''Create the batches for the training and validation data'''
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]
print(len(train_vects), len(train_vects[1]))
print(len(y_train))
print(y_train[0])

num_folds = 5
kFold = 0

X_train_folds = np.array_split(train_vects[:data_size], num_folds)
y_train_folds = np.array_split(y_train[:data_size], num_folds)

x_train, x_valid = [], list(X_train_folds[kFold])
y_train, y_valid = [], list(y_train_folds[kFold])
for i in range(0,num_folds):
    if i != kFold:
        x_train.extend(X_train_folds[i])
        y_train.extend(y_train_folds[i])


print('x_train = ',len(x_train),len(x_train[1]),len(x_train[1][0]))
def check_dist(a):
  return np.array([np.percentile(a,i) for i in [0,25,50,75,100]])
def np_distances(cents,points):
  p1 = np.tile(points, (cents.shape[0], 1)).reshape(cents.shape[0],points.shape[0],points.shape[1])
  c1 = np.expand_dims(cents,axis=1)
  d1 = p1 - c1
  d2 = d1*d1
  d3 = np.sum(d2,axis=2)
  return d3
def match_sents(k_cents,all_sents):
  # res = []
  dProsSents = np_distances(k_cents,np.array(all_sents))
  dProsSentsArgSort = np.argsort(dProsSents,axis=1)
  return all_sents[dProsSentsArgSort[:,0]]

print(check_dist([-5,2,10,0,1,5,4,6,7,-4,-3,8]))
print(check_dist([-1,0.5,1,0.7,-0.8,-0.5,-0.2,0.8, 0.3]))
# print('y_train: ',y_train.shape)

30001 5
30001
[1]
x_train =  2400 2 768
[-5.   -0.75  3.    6.25 10.  ]
[-1.  -0.5  0.3  0.7  1. ]


In [168]:
#----clustering-----
all_sents = [sent for para in x_train for sent in para]
k_protos = 20
print(len(all_sents),len(all_sents[0]))
kmedoids = KMedoids(n_clusters=k_protos,random_state=0).fit(all_sents[0:10000])
# kmedoids = KMedoids(n_clusters=k_protos,metric='cosine',random_state=0).fit(all_sents[0:10000])
k_cents = kmedoids.cluster_centers_
print(k_cents.shape)

14953 768
(20, 768)


In [169]:
for c in k_cents:
  print(check_dist(c))

[-2.21362066 -0.36394682 -0.00421334  0.35230722  2.2496078 ]
[-2.24929309 -0.36212804  0.01551494  0.321096    1.78031921]
[-1.85124207 -0.34618915  0.00813584  0.33220163  2.2175765 ]
[-3.07907462e+00 -3.41143712e-01 -2.45070679e-03  3.15083027e-01
  1.84032619e+00]
[-1.86753654 -0.386037   -0.00364652  0.36949214  1.88630879]
[-1.89380932e+00 -4.50098388e-01 -2.30933505e-03  3.88980456e-01
  2.44710279e+00]
[-3.03781986 -0.3639546  -0.02061732  0.33252376  2.41140342]
[-2.53508568 -0.3230486  -0.0069954   0.35568865  2.1769278 ]
[-2.37569451 -0.37786707 -0.00452666  0.36098225  2.08828235]
[-2.32970691 -0.36985804 -0.01914496  0.31179453  2.02894163]
[-2.5167172  -0.34546048  0.01047673  0.35073632  1.87700129]
[-1.77577424 -0.33705235 -0.03569043  0.30009926  1.81651294]
[-2.23104453e+00 -3.33338164e-01  2.06786237e-03  3.21544953e-01
  1.72285604e+00]
[-2.06194615 -0.38023037  0.01769366  0.35381508  1.52218342]
[-2.70944953e+00 -3.49739827e-01 -3.51661947e-04  3.37845221e-01
  2.

#Build and train model


In [0]:
def lstm_cell(lstm_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    return drop

In [171]:
# The default parameters of the model
# n_words = len(word_index)
n_words = 1
embed_size = 300
batch_size = 1
lstm_size = 128
num_layers = 2
dropout = 0.5
learning_rate = 0.01
epochs = 100
multiple_fc = False
fc_units = 256
vect_size = 768

tf.reset_default_graph()

# Declare placeholders we'll feed into the graph
with tf.name_scope('inputs'):
    inputs = tf.placeholder(tf.float32, [batch_size, None, vect_size], name='inputs')
    print('[db] inputs = ', inputs)

with tf.name_scope('labels'):
    labels = tf.placeholder(tf.int32, [batch_size, None], name='labels')

    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Build the RNN layers
with tf.name_scope("RNN_layers"):
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(lstm_size, keep_prob) for _ in range(num_layers)])

# Set the initial state
with tf.name_scope("RNN_init_state"):
    initial_state = cell.zero_state(batch_size, tf.float32)

# prototypes
with tf.name_scope("prototypes"):
    protos_values = tf.placeholder(tf.float32, [k_protos, vect_size], name='protos_values')

    prototypes = tf.get_variable('prototypes', shape=[k_protos, vect_size],
                                 initializer=tf.constant_initializer(k_cents[:k_protos]), trainable=True)
    print('[db] prototypes = ', prototypes)

    proto_assign_op = prototypes.assign(protos_values)
    print('[db] proto_assign_op = ', proto_assign_op)



def compute_distance(outputs, p, n_protos):
    tmp1 = tf.expand_dims(outputs, 2)
    tmp1 = tf.broadcast_to(tmp1, [tmp1.shape[0].value, tf.shape(tmp1)[1], n_protos, vect_size])

    tmp2 = tf.broadcast_to(p, [tmp1.shape[0].value, tf.shape(tmp1)[1], n_protos, vect_size])

    tmp3 = tmp1 - tmp2
    tmp4 = tmp3 * tmp3
    distances = tf.reduce_sum(tmp4, axis=3)
    return distances, tf.reduce_min(distances, axis=1)

def e_func(x,a = 0.1, e=2.7182818284590452353602874713527):
    return tf.math.pow(e,-(a*x))

full_distances, full_min_dist = compute_distance(inputs, prototypes, k_protos)


min_dist_ind = tf.nn.softmax(-full_distances  * beta)

dist_hot_vect = min_dist_ind * e_func(tf.math.sqrt(full_distances + 1e-8))

with tf.name_scope("RNN_forward"):

    outputs, final_state = tf.nn.dynamic_rnn(cell, dist_hot_vect, initial_state=initial_state)


# Create the fully connected layers
with tf.name_scope("fully_connected"):
    # Initialize the weights and biases
    weights = tf.truncated_normal_initializer(stddev=0.1)
    biases = tf.zeros_initializer()

    dense = tf.contrib.layers.fully_connected(outputs[:, -1], num_outputs=fc_units, activation_fn=tf.sigmoid,
                                              weights_initializer=weights, biases_initializer=biases)
    dense = tf.contrib.layers.dropout(dense, keep_prob)

    # Depending on the iteration, use a second fully connected layer
    if multiple_fc == True:
        dense = tf.contrib.layers.fully_connected(dense, num_outputs=fc_units, activation_fn=tf.sigmoid,
                                                  weights_initializer=weights, biases_initializer=biases)
        dense = tf.contrib.layers.dropout(dense, keep_prob)

# Make the predictions
with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(dense,
                                                    num_outputs=2,
                                                    activation_fn=tf.sigmoid,
                                                    weights_initializer=weights,
                                                    biases_initializer=biases)
    predictions = predictions + 1e-8
    tf.summary.histogram('predictions', predictions)

# Calculate the cost
with tf.name_scope('cost'):
    
    def tight_pos_sigmoid_offset(x, offset, e=2.7182818284590452353602874713527):
	      return 1 / (1 + tf.math.pow(e, (1 * (offset*x-0.5))))


    squared_cost = tf.losses.mean_squared_error(labels, predictions) + 1e-8

    def pw_distance(A):
        # A = tf.constant([[1, 1], [4, 4], [5, 5],[6,6]])
        r = tf.reduce_sum(A * A, 1)
        print('[db] r = ', r)
        # turn r into column vector
        r = tf.reshape(r, [-1, 1])
        D = r - 2 * tf.matmul(A, tf.transpose(A)) + tf.transpose(r)
        return D


    costR_d = tf.reduce_sum(tf.reduce_min(full_distances,axis=2))
    
    d = pw_distance(prototypes ) 
    
    diag_ones = tf.convert_to_tensor(np.eye(pos_pro_k + neg_pro_k, dtype=float))
    diag_ones = tf.dtypes.cast(diag_ones, tf.float32)
    d1 = d + diag_ones * tf.reduce_max(d) 
    d2 = tf.reduce_min(d1, axis=1) 
    min_d2_dist = tf.reduce_min(d2) 
    costC = tight_pos_sigmoid_offset(min_d2_dist, 1) + 1e-8

    cost = squared_cost + 0.0001 * costR_d + 0.1 * costC


    
    # test_var = [full_distances,e_func(full_distances),dist_hot_vect]
    test_var = [costC]


# Train the model
with tf.name_scope('train'):
    lr1, lr2 = 0.0001, 0.0005
    optimizer = tf.train.AdamOptimizer(lr1).minimize(cost)

    # optimizer1 = tf.train.AdamOptimizer(lr1).minimize(cost1)
    # optimizer2 = tf.train.AdamOptimizer(lr2).minimize(cost2)

# Determine the accuracy
with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
        all_labels_true = tf.reduce_min(tf.cast(correct_pred, tf.float32),1)
        # accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        accuracy = tf.reduce_mean(all_labels_true)




[db] inputs =  Tensor("inputs/inputs:0", shape=(1, ?, 768), dtype=float32)
[db] prototypes =  <tf.Variable 'prototypes_1:0' shape=(20, 768) dtype=float32_ref>
[db] proto_assign_op =  Tensor("prototypes/Assign:0", shape=(20, 768), dtype=float32_ref)
[db] r =  Tensor("cost/Sum_1:0", shape=(20,), dtype=float32)


In [174]:
saver = tf.train.Saver()
log_string = 'ru={},fcl={},fcu={}'.format(lstm_size,
                                                      multiple_fc,
                                                      fc_units)   
recorded_train_acc, recorded_valid_acc = [], []     
with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        valid_loss_summary = []
        
        # Keep track of which batch iteration is being trained
        iteration = 0

        avg_train_acc = 0
        
        with open('_prototypes', 'wb') as fp:
          pickle.dump(k_cents, fp)
        with open ('_prototypes', 'rb') as fp:
          _prototypes = pickle.load(fp)
              
        for e in range(epochs):
            state = sess.run(initial_state)
            
            # Record progress with each epoch
            train_loss, train_acc, val_acc, val_loss = [], [], [], []
            # optimizer = optimizer1
            print('[db] Using opt1')
            # if avg_train_acc > 0.65:
            #   print('[db] Using opt2')
            #   optimizer = model.optimizer2

            with tqdm(total=len(x_train)) as pbar:
                for i, (x, y) in enumerate(get_batches(x_train, y_train, batch_size), 1):

                    feed = {inputs: x,
                            protos_values: _prototypes,      
                            labels: y,
                            keep_prob: dropout,
                            initial_state: state,
                            }
                    loss, \
                    acc, \
                    state,\
                    _, \
                    _prototypes, \
                    _predictions, \
                    _test_var = sess.run([
                                                             cost, 
                                                             accuracy, 
                                                             final_state, 
                                                             optimizer,
                                                             prototypes,
                                                             predictions,
                                                             test_var
                                                            ], 
                                                           feed_dict=feed)
    
                    if i % 1000 == 0:
                      # print("test_var: ",i)
                      for v in _test_var:
                        print('v = ',v,v.shape)
                      with open('./test_var', 'wb') as fp:
                        pickle.dump(_test_var, fp)
                      # print("_neg_prototypes : ",_neg_prototypes)
                    # Record the loss and accuracy of each training batch
                    train_loss.append(loss)
                    train_acc.append(acc)                  
                    # Record the progress of training
                    
                    iteration += 1
                    pbar.update(batch_size)

            # --- projection
            if e % 10 == 0:
              new_protos = match_sents(_prototypes,np.array(all_sents[:10000]))

              _prototypes, _prototypes_values, _  = sess.run([prototypes, protos_values, proto_assign_op], feed_dict={protos_values: new_protos})

              print(_prototypes)
              print('Saving projected protos...')
              with open('_prototypes', 'wb') as fp:
                    pickle.dump(_prototypes, fp)

              with open("./drive/My Drive/deep_learning/dl_research/interpretable_RNN/vars/_prototypes", 'wb') as fp:
                    pickle.dump(_prototypes, fp)
              checkpoint = "./sentiment_{}.ckpt".format(log_string)
              saver.save(sess, checkpoint)
            # -----

            # Average the training loss and accuracy of each epoch
            avg_train_loss = np.mean(train_loss)
            avg_train_acc = np.mean(train_acc)
            
            recorded_train_acc.append(avg_train_acc)

            val_state = sess.run(initial_state)
            with tqdm(total=len(x_valid)) as pbar:
                for x, y in get_batches(x_valid, y_valid, batch_size):

                    feed = {inputs: x,
                            protos_values: _prototypes,
                            # model.labels: np.squeeze(y,axis=2),
                            labels: y,
                            keep_prob: 1,
                            initial_state: val_state,
                            # model.pretrained_embedding: embedding_matrix
                            }
                    batch_loss, \
                    batch_acc, \
                    val_state, \
                    _prototypes, \
                    _predictions  = sess.run([ 
                                                                          cost, 
                                                                          accuracy, 
                                                                          final_state,
                                                                          prototypes,
                                                                          predictions
                                                                          ], 
                                                                         feed_dict=feed)
                    
                    # Record the validation loss and accuracy of each epoch
                    val_loss.append(batch_loss)
                    val_acc.append(batch_acc)

                    pbar.update(batch_size)
            
            # Average the validation loss and accuracy of each epoch
            avg_valid_loss = np.mean(val_loss)    
            avg_valid_acc = np.mean(val_acc)

            recorded_valid_acc.append(avg_valid_acc)


            valid_loss_summary.append(avg_valid_loss)
            
            # Record the validation data's progress

            print("Epoch: {}/{}".format(e, epochs))
            print("Train Loss: {:.3f}".format(avg_train_loss))
            print("Train Acc: {:.3f}".format(avg_train_acc))
            print("Valid Loss: {:.3f}".format(avg_valid_loss))
            print("Valid Acc: {:.3f}".format(avg_valid_acc))

            with open('_recorded_train_acc', 'wb') as fp:
                    pickle.dump(recorded_train_acc, fp)
            with open('_recorded_valid_acc', 'wb') as fp:
                    pickle.dump(recorded_valid_acc, fp)

            # Stop training if the validation loss does not decrease after 3 epochs
            if avg_valid_loss > min(valid_loss_summary):
                print("No Improvement.")
                stop_early += 1
                if stop_early == 100:
                    break   
            
            # Reset stop_early if the validation loss finds a new low
            # Save a checkpoint of the model
            else:
                print("New Record!")
                stop_early = 0
                checkpoint = "./sentiment_{}.ckpt".format(log_string)
                saver.save(sess, checkpoint)

  0%|          | 1/2400 [00:00<06:25,  6.22it/s]

[db] Using opt1


 42%|████▏     | 1009/2400 [00:11<00:15, 87.33it/s]

v =  1e-08 ()


 84%|████████▍ | 2012/2400 [00:23<00:04, 96.67it/s]

...

Epoch: 12/100
Train Loss: 0.233
Train Acc: 0.796
Valid Loss: 0.233
Valid Acc: 0.822
No Improvement.
[db] Using opt1


 42%|████▏     | 1008/2400 [00:11<00:17, 81.57it/s]

v =  1e-08 ()


 48%|████▊     | 1141/2400 [00:13<00:14, 83.94it/s]


KeyboardInterrupt: ignored

#Show the prototypes

In [0]:
with open ('./_prototypes', 'rb') as fp:
	_prototypes = pickle.load(fp)


In [176]:
print('protos: ',_prototypes.shape)
for p in _prototypes:
  print(check_dist(p))
print('y_train_full: ',y_train_full.shape)

protos:  (20, 768)
[-2.21362066 -0.36394682 -0.00421334  0.35230722  2.2496078 ]
[-2.24929309 -0.36212804  0.01551494  0.321096    1.78031921]
[-1.85124207 -0.34618915  0.00813584  0.33220163  2.2175765 ]
[-3.07907462e+00 -3.41143712e-01 -2.45070679e-03  3.15083027e-01
  1.84032619e+00]
[-1.86753654 -0.386037   -0.00364652  0.36949214  1.88630879]
[-1.89380932e+00 -4.50098388e-01 -2.30933505e-03  3.88980456e-01
  2.44710279e+00]
[-3.03781986 -0.3639546  -0.02061732  0.33252376  2.41140342]
[-2.53508568 -0.3230486  -0.0069954   0.35568865  2.1769278 ]
[-2.37569451 -0.37786707 -0.00452666  0.36098225  2.08828235]
[-2.32970691 -0.36985804 -0.01914496  0.31179453  2.02894163]
[-2.5167172  -0.34546048  0.01047673  0.35073632  1.87700129]
[-1.77577424 -0.33705235 -0.03569043  0.30009926  1.81651294]
[-2.23104453e+00 -3.33338164e-01  2.06786237e-03  3.21544953e-01
  1.72285604e+00]
[-2.06194615 -0.38023037  0.01769366  0.35381508  1.52218342]
[-2.70944953e+00 -3.49739827e-01 -3.51661947e-04  

In [177]:
print(len(x_train),len(y_train))

2400 2400


In [178]:
p_count = 0
d_pos = {}
for p_count,p in enumerate(_prototypes):
  print('p_count = ',p_count)
  s_count = 0
  d_pos[p_count] = {}
  for i,para in enumerate(train_vects[:data_size]):
    for j,s in enumerate(para):
      if len(train_clean[i].split(".")[j]) < 5:
        continue
      d_pos[p_count][(i,j)] = np.linalg.norm(train_vects[i][j]-p)
      # print('i,j = ',i,j)
      # d_pos[p_count][(i,j)] = spatial.distance.cosine(train_vects[i][j], p)
      s_count+=1
  print('count = ',s_count)

p_count =  0
count =  18650
p_count =  1
count =  18650
p_count =  2
count =  18650
p_count =  3
count =  18650
p_count =  4
count =  18650
p_count =  5
count =  18650
p_count =  6
count =  18650
p_count =  7
count =  18650
p_count =  8
count =  18650
p_count =  9
count =  18650
p_count =  10
count =  18650
p_count =  11
count =  18650
p_count =  12
count =  18650
p_count =  13
count =  18650
p_count =  14
count =  18650
p_count =  15
count =  18650
p_count =  16
count =  18650
p_count =  17
count =  18650
p_count =  18
count =  18650
p_count =  19
count =  18650


In [179]:
import operator
protos_sents = {}
k_closest_sents = 5
recorded_protos_score = {}
for l in range(pos_pro_k + neg_pro_k):
  print("prototype index = ",l)
  recorded_protos_score[l] = {}
  sorted_d = sorted(d_pos[l].items(), key=operator.itemgetter(1))
  proto_scores = []
  closest_sents = []
  for k in range(k_closest_sents):
    (i,j) = sorted_d[k][0]
    print(sorted_d[k], y_train_full[i], train_not_clean[i].split(".")[j])
    # print()
    proto_scores.append(int(y_train_full[i]))
    # closest_sents.append(train_not_clean[i].split(".")[j] + " " + str(y_train_full[i]))
  (m,n) = sorted_d[0][0] 
  protos_sents[l] = (train_not_clean[m].split(".")[n],y_train_full[i],sum(proto_scores)/len(proto_scores),closest_sents)

prototype index =  0
((2135, 2), 0.0) 0  Even when it did work it was a hit or miss thing
((272, 7), 10.323638) 0 95 above and beyond the already rather pricy product
((701, 3), 10.72252) 0  I did, although it was unrewarding to do so
((2746, 0), 10.749016) 1 This is made well and strudy, but will wear out
((1367, 5), 10.792902) 0  Once it was on it really didn't do what it is supposed to do - it's more awkward than anything
prototype index =  1
((1581, 4), 0.0) 0 While certain parts of this story are interesting, in it's entirety, i would have to give it two thumbs down (
((1339, 2), 9.720231) 0 You'll find a couple of interesting ideas, a lot of trite and derivative ones and occasionally some baffling lack of coherent character development
((2617, 5), 9.727579) 0  But considering the ending seemed to be rather ill-fitting to the rest of the tale, I'll have to pass on giving it a high rating
((102, 5), 10.011818) 1  While she sometimes seems too guillible, I have to remember that it i